# Финальный проект: вариант 2

## Задание 1. A/B–тестирование

In [2]:
import pandas as pd 
import requests 
from urllib.parse import urlencode
import numpy as np
from sklearn.preprocessing import OneHotEncoder as SklearnOneHotEncoder
from statsmodels.regression.linear_model import OLS

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(
    font_scale=2,
    style="whitegrid",
    rc={'figure.figsize':(20,7)}
        )

#### Загружаем данные
Таблица с группами **groups.csv** 

### **(!)** Если будет ошибка со стороны Яндекса "превышен лимит на скачивание файла", то подгрузим файл локальный

In [11]:

# Получаем таблицу с группами
# используем api 
base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?' 
public_key = 'https://disk.yandex.ru/d/hmRKKZTCIr3gnQ' 

# получаем url 
final_url = base_url + urlencode(dict(public_key=public_key)) 
response = requests.get(final_url) 
download_url_orders = response.json()['href'] 

#загружаем файл в df 
download_response = requests.get(download_url_orders) 
groups_df = pd.read_csv(download_url_orders, sep=';')

In [12]:
groups_df.head()

,id,grp
0,1489,B
1,1627,A
2,1768,B
3,1783,B
4,1794,A


Доп.файл по группам **groups_add.csv** 

In [ ]:
try:
    # Получаем доп.таблицу с группами
    # используем api 
    base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?' 
    public_key = 'https://disk.yandex.ru/d/5Kxrz02m3IBUwQ' 

    # получаем url 
    final_url = base_url + urlencode(dict(public_key=public_key)) 
    response = requests.get(final_url) 
    download_url_orders = response.json()['href'] 

    # загружаем файл в df 
    download_response = requests.get(download_url_orders) 
    groups_add_df = pd.read_csv(download_url_orders, sep=',')
except:
    path_groups_add = 'file_groups_add_error.csv'
    groups_add_df = pd.read_csv(path_groups_add, sep=',')

In [ ]:
 groups_add_df.head()

Таблица с информацией о пользователях, которые зашли на платформу в дни проведения эксперимента **active_studs.csv** 

In [ ]:
try:
    # Получаем файл с информацией о пользователях, которые зашли на платформу в дни проведения эксперимента. 
    # используем api 
    base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?' 
    public_key = 'https://disk.yandex.ru/d/Tbs44Bm6H_FwFQ' 

    # получаем url 
    final_url = base_url + urlencode(dict(public_key=public_key)) 
    response = requests.get(final_url) 
    download_url_orders = response.json()['href'] 

    # загружаем файл в df 
    download_response = requests.get(download_url_orders) 
    active_studs_df = pd.read_csv(download_url_orders, sep=',') 
except:
    path_active_studs = 'active_studs_error'
    active_studs_df = pd.read_csv(path_active_studs, sep=',')

In [ ]:
active_studs_df.head(5)

Таблица с информацией об оплатах пользователей в дни проведения эксперимента **checks.csv**

In [ ]:
try:
    # Получаем файл с информацией об оплатах пользователей в дни проведения эксперимента.
    # используем api 
    base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?' 
    public_key = 'https://disk.yandex.ru/d/pH1q-VqcxXjsVA' 

    # получаем url 
    final_url = base_url + urlencode(dict(public_key=public_key)) 
    response = requests.get(final_url) 
    download_url_orders = response.json()['href'] 

    # загружаем файл в df 
    download_response = requests.get(download_url_orders) 
    checks_df = pd.read_csv(download_url_orders, sep=';') 
except:
    path_checks = 'checks_error'
    checks_df = pd.read_csv(path_checks, sep=';')

In [ ]:
checks_df.head(5)

### Data Validation

1. Проверим на null и тип данных таблицы группы клиентов

In [ ]:
groups_df.info()

In [ ]:
groups_add_df.info()

In [ ]:
active_studs_df.info()

In [ ]:
# Проверяем таблицу с участниками эксперимента на дубликаты
active_studs_df.loc[active_studs_df.duplicated()]

In [ ]:
checks_df.info()

In [ ]:
checks_df.describe()

### Собираем общий df  
#### 1. Объединим таблицы с группами, и создадим полный справочник клиентов

In [ ]:
groups_full_df = groups_df.append(groups_add_df)
groups_full_df.shape

In [ ]:
# 1.1. Проверяем на дубликаты
groups_full_df.loc[groups_full_df.duplicated()]

In [ ]:
# 1.2. Оценим соотношение людей в группах
count_persosons = sns.countplot(x='grp', data = groups_full_df).set(title='Количество людей в группах')

#### 2. Добавляем столбец info в таблицу с активными пользователями, чтобы при объединении со справочником всех пользоваталей была возможность отфильтровать активных юзеров.

In [ ]:
active_studs_df['info'] = 'были на платформе'
active_studs_df.head()

#### 3. Объединяем справочник с клиентами и таблицу с активностью

In [ ]:
# Создаем таблицу всех юзеров с группой тестирования и инфой об активности.
all_users = pd.merge(groups_full_df, active_studs_df, how ='left', left_on='id', right_on='student_id').drop('student_id', axis=1)
all_users.head()

#### 4. Получаем общий df

In [ ]:
# 4.1. К таблице с чеками добавляем юзеров, которые были на платформе в день эксперимента.
big_df = pd.merge(checks_df, all_users, how ='outer',left_on='student_id', right_on='id').drop('student_id', axis=1)
big_df = big_df.query('info=="были на платформе" or rev>0').set_index('id')
big_df['rev'] = big_df['rev'].fillna(0)
# 4.2. Создаем столбец -лейбл, который принимает значение 1, если пользователь совершил покупку и 0 если покупку не совершал.
big_df['converted'] = np.where(big_df.rev>0,1,0)
big_df = big_df.reset_index()
experiment_2_df = big_df
big_df.head()

### EDA 

#### 1. Подготовим df для анализа чеков по группам

In [ ]:
# Переведем чеки в целое число и сгруппируем, чтобы посмотреть сколько вариантов оплат существует для пользователей в разных механиках.
vs_checks_df = big_df.astype({'rev':'int'})
a_checks = vs_checks_df.query('grp=="A" & rev>0').rev.value_counts().to_frame().reset_index().rename(columns={'index':'check','rev':'clients_a'}).sort_values('check')
b_checks = vs_checks_df.query('grp=="B" & rev>0').rev.value_counts().to_frame().reset_index().rename(columns={'index':'check','rev':'clients_b'}).sort_values('check')
# Разбиваем чеки группы А на квантили
a_checks = a_checks.assign(quan_check_a = pd.qcut(a_checks['check'], q=4,duplicates='drop'))
b_checks = b_checks.assign(quan_check_b = pd.qcut(b_checks['check'], q=4,duplicates='drop'))
# Объединяем таблицы, чтобы узнать какие чеки наиболее популярны в группах
a_b_checks = a_checks.merge(b_checks, how='outer',on='check')
# Считаем общее количество клиентов, чтобы высчитать %
a_buyers = a_b_checks.clients_a.sum()
b_buyers = a_b_checks.clients_b.sum()
# считаем % клиентов по группам, по чекам
a_b_checks['clients_a_%'] = np.round(((a_b_checks.clients_a/a_buyers)*100),2)
a_b_checks['clients_b_%'] = np.round(((a_b_checks.clients_b/b_buyers)*100),2)
a_b_checks.head()

#### 2. Узнаем какой % клиентов какие чеки оставляет.

In [ ]:
# Оставляем только нужные столбцы для графика
for_bar_pic_df = a_b_checks[['check','clients_a_%','clients_b_%']].set_index('check')
plt.figure(figsize=(20,6))
# Строим график
ab_checks_pic = for_bar_pic_df.plot(kind='bar', stacked=False, color=['#4c72b0', '#dd8452'])
plt.title('% клиентов, купивших по чекам')
plt.xlabel('check')
plt.ylabel('Percentage');

Видно, что в группе 'B' большая часть чеков (более 30%) приходится на 1900 руб. и ощущется смещение в сторону увеличения чека.  также вариантов оплаты больше. 

#### 3. Узнаем количество вариантов оплаты по группам. (чем отличаются механики)

In [ ]:
payment_variants = vs_checks_df.query('rev>0 ').groupby('grp').agg({'rev':'nunique'}).reset_index()
sns.barplot(data= payment_variants, x='grp', y='rev').set(title='Кол-во вариантов оплаты в группах');

#### 4. вариантов оплаты больше 20 в обеих группах, поэтому поделим чеки на сегменты для дальнейшего удобства анализа и посчитаем % клиентов по сегментам чеков.

In [ ]:
# Разделяем df на группы для barplot-ов 
qan_check_a_bar = a_b_checks.groupby('quan_check_a')['clients_a_%'].sum().to_frame().reset_index()
qan_check_b_bar = a_b_checks.groupby('quan_check_b')['clients_b_%'].sum().to_frame().reset_index()

sns.barplot(data = qan_check_a_bar, x='quan_check_a', y='clients_a_%').set(title='% клиентов в диапазонах чеков (группа А)');

In [ ]:
sns.barplot(data = qan_check_b_bar, x='quan_check_b', y='clients_b_%').set(title='% клиентов в диапазонах чеков (группа В)');

В группе "B" больший % клиентов смещен к высоким чекам, чем в группе "A"

#### 5. Построим boxplot, увидеть распредление более наглядно

In [ ]:
checks_box_pic = sns.catplot( data= big_df[big_df['rev']>0], x="grp", y="rev", kind="box").set(title='Распределение чеков, платящих клиентов, по группам')

Рассмотрим **Median Revenue Per Paying User (MRPPU)** - средний платеж на всех платящих пользователей.

In [ ]:
mrppu_by_groups = big_df.query('rev>0').reset_index().groupby(['grp'])\
                .agg(MRPPU = ('rev','median'))
mrppu_by_groups

#### 6. Сделаем сводный df, чтобы увидеть основные показатели (cr%, ARPU, ARPPU, MRPPU) по группам

In [ ]:

# 3.1. Сгруппируем данные так, чтобы увидеть какова была конверсия для обеих групп.
groups = big_df.reset_index().groupby(['grp'])\
                .agg(\
                    ids = ('id','size'),\
                    revenue = ('rev','sum'),\
                    converted =('converted','sum')\
                    )
# Добавляем столбец с MRPPU
groups = groups.join(mrppu_by_groups)
#  Считаем клиентов, которые не делали покупок
groups['unconverted'] = groups['ids']-groups['converted']
# Считаем % клиентов, которые не делали покупок
groups['part_unconv_%'] = np.round(((groups['unconverted']/groups['ids'])*100),2)
# Считаем CR% 
groups['cr_%'] = round(((groups.converted/groups.ids)*100),2)
groups['revenue'] = np.round((groups.revenue),2)
# # Посчитаем разницу в CR между группами в %
groups['diff%_cr_B-A'] = np.round(((groups.loc['B']['cr_%'] - groups.loc['A']['cr_%'])/groups.loc['A']['cr_%']),2)
# Посчитаем разницу в ARPU между группами абсолют. и в %
groups['ARPU'] = np.round((groups.revenue/groups.ids),2)
groups['diff%_ARPU_B-A'] = np.round((((groups.loc['B']['ARPU'] - groups.loc['A']['ARPU']) / groups.loc['A']['ARPU'])*100),2)
# Посчитаем разницу в ARPPU между группами абсолют. и в %
groups['ARPPU'] = np.round((groups.revenue/groups.converted),2)
# groups['diff_ARPPU_B-A'] = np.round((groups.loc['B']['ARPPU'] - groups.loc['A']['ARPPU']),2)
groups['diff%_ARPPU_B-A'] = np.round((((groups.loc['B']['ARPPU'] - groups.loc['A']['ARPPU']) / groups.loc['A']['ARPPU'])*100),2)
groups['diff%_MRPPU_B-A'] = np.round((((groups.loc['B']['MRPPU'] - groups.loc['A']['MRPPU']) / groups.loc['A']['MRPPU'])*100),2)
# Посчитаем % клиентов в группах
groups['%_clients'] = np.round(((groups['ids']/(groups.loc['A']['ids'] + groups.loc['B']['ids']))*100),2)
groups = groups.reset_index()
summary_deviation = groups[['grp','%_clients','cr_%','ARPU','ARPPU','MRPPU']]
summary_deviation

#### 7. Формируем табличку с изменениями по основным метрикам (в %) по группам

In [ ]:
summary_deviation_a_b = groups[[ 'diff%_cr_B-A','diff%_ARPU_B-A','diff%_ARPPU_B-A', 'diff%_MRPPU_B-A']]
a_vs_b_deviation = summary_deviation_a_b.stack().to_frame().reset_index().query('level_0==0')\
                    .rename(columns={'level_0':'index', 'level_1':'metric'})
a_vs_b_deviation = a_vs_b_deviation.rename(columns= {a_vs_b_deviation.columns[2]:'B_vs_A_%'})
a_vs_b_deviation['A'] = 100 + a_vs_b_deviation['B_vs_A_%']
a_vs_b_deviation['B'] = 100 - a_vs_b_deviation['B_vs_A_%']
a_vs_b_deviation[['metric','B_vs_A_%']]

Как видно из таблицы изменение в конверсии между группами не значительно, а показатели ARPU (+18%),ARPPU (+28%), и MRPPU (+35%) в пользу тестовой группы

#### 8. Отразим на графике изменения по основным метрикам (в %) по группам

In [ ]:
for_stacked_deviation = a_vs_b_deviation[['metric','A','B']].set_index('metric')
for_stacked_deviation.plot(kind='bar', stacked=False, color=['#4c72b0', '#dd8452']);

Осталось проверить являются ли эти изменения статистически значимыми

 ## Проверка гипотез 
###  I Фиксируем гипотезы (CR)
**H0** - новая механика оплаты услуг на сайте НЕ влияет на конверсию в покупку <br>
**H1** - новая механика оплаты услуг на сайте влияет на конверсию в покупку <br> 
<br> Уровень значимости p= 0.05 <br>

#### 1. Готовим таблицу для хи-теста 


In [ ]:
# Оставляем только нужные столбцы из df
experiment_2_df = experiment_2_df[['grp','converted']]
# 1) разделяем группы на два отдельных датафрейма
a = experiment_2_df[experiment_2_df['grp'] == 'A']
b = experiment_2_df[experiment_2_df['grp'] == 'B']
# 2) A-purchase, A-nopurchase, B-purchase, B-nopurchase
a_purchase = a.converted.sum()
a_nopurchase = a.converted.size - a.converted.sum()
b_purchase = b.converted.sum()
b_nopurchase = b.converted.size - b.converted.sum()
# 3) Создаем массив
T = np.array([[a_purchase, a_nopurchase], [b_purchase, b_nopurchase]])

In [ ]:
# 1.3. Запускаем тест
import scipy
from scipy import stats
print(scipy.stats.chi2_contingency(T,correction=False)[1])

### Вывод по тесту: 
Значение р было рассчитано на уровне 40%. При 5-процентный уровене значимости, мы можем сделать вывод, что p-значение больше альфы и что мы не отвергаем нулевую гипотезу. Проще говоря, нет никакой разницы в конверсиях между старой и новой механикой оплаты.

### II Фиксируем гипотезы (Median Revenue Per Paying User) <br>
**H0** - новая механика оплаты услуг на сайте НЕ влияет на MRPPU <br>
**H1** - новая механика оплаты услуг на сайте влияет на MRPPU <br> 
<br> Уровень значимости p= 0.05 <br>

#### Посчитаем p-value тремя способами: t-тест, U-тест, бутстрап средних

In [ ]:
# p-value U-test
from scipy.stats import mannwhitneyu

mann_result = mannwhitneyu(big_df.query('rev>0 & grp=="A"')['rev'], big_df.query('rev>0 & grp=="B"')['rev'])
mann_result

**Результат:** p_value<0.05 Распределения значимо отличаются

In [ ]:
# p-value T-test
from scipy import stats

t_test_result = stats.ttest_ind(big_df.query('rev>0 & grp=="A"')['rev'], big_df.query('rev>0 & grp=="B"')['rev'])
t_test_result

**Результат:** p_value < 0.05 , отклоняем нулевую гипотезу. Различия стат. значимы

In [ ]:
# p-value bootstrap

from scipy.stats import norm
import matplotlib.pyplot as plt

from tqdm.auto import tqdm

plt.style.use('ggplot')
def get_bootstrap(
    data_column_1, # числовые значения первой выборки
    data_column_2, # числовые значения второй выборки
    boot_it = 1000, # количество бутстрэп-подвыборок
    statistic = np.median, # интересующая нас статистика
    bootstrap_conf_level = 0.95 # уровень значимости
):
    boot_data = []
    for i in tqdm(range(boot_it)): # извлекаем подвыборки
        samples_1 = data_column_1.sample(
            len(data_column_1), 
            replace = True # параметр возвращения
        ).values
        
        samples_2 = data_column_2.sample(
            len(data_column_1), 
            replace = True
        ).values
        
        boot_data.append(statistic(samples_1)-statistic(samples_2)) # mean() - применяем статистику
        
    pd_boot_data = pd.DataFrame(boot_data)
        
    left_quant = (1 - bootstrap_conf_level)/2
    right_quant = 1 - (1 - bootstrap_conf_level) / 2
    quants = pd_boot_data.quantile([left_quant, right_quant])
        
    p_1 = norm.cdf(
        x = 0, 
        loc = np.mean(boot_data), 
        scale = np.std(boot_data)
    )
    p_2 = norm.cdf(
        x = 0, 
        loc = -np.mean(boot_data), 
        scale = np.std(boot_data)
    )
    p_value = min(p_1, p_2) * 2
        
    # Визуализация
    _, _, bars = plt.hist(pd_boot_data[0], bins = 50)
    for bar in bars:
        if bar.get_x() <= quants.iloc[0][0] or bar.get_x() >= quants.iloc[1][0]:
            bar.set_facecolor('red')
        else: 
            bar.set_facecolor('grey')
            bar.set_edgecolor('black')
    
    plt.style.use('ggplot')
    plt.vlines(quants,ymin=0,ymax=50,linestyle='--')
    plt.xlabel('boot_data')
    plt.ylabel('frequency')
    plt.title("Histogram of boot_data")
    plt.show()
       
    return {"boot_data": boot_data, 
            "quants": quants, 
            "p_value": p_value}

In [ ]:
booted_data = get_bootstrap(big_df.query('rev>0 & grp=="B"')['rev'], big_df.query('rev>0 & grp=="A"')['rev']) # в результате хранится разница двух распределений, ДИ и pvalue

In [ ]:
# p-value bootstrap
p_value_bootstrap = booted_data["p_value"] # альфа
p_value_bootstrap

In [ ]:
booted_data["quants"]

**Результат:** p_value < 0.05 , отклоняем нулевую гипотезу. Различия в медианах стат. значимы. Доверительный интервал не пересекает 0 это означает, что нулевое различие не согласуется в полученными нами данными. А это прямое свидетельство в пользу того, что различие между медианами ненулевое. Таким образом разница в средних между группами находится в диапазоне (150 - 400)

#### 1. Проверим данные на соответствия требованиям применения t-testа:
##### 1.1. требование к нормальности данных обеих групп при применении t-теста.

Построим графики распределния

In [ ]:
sns.distplot(big_df.query('rev>0 & grp=="A"')['rev'])
sns.distplot(big_df.query('rev>0 & grp=="B"')['rev']);

Ярко выражена ассиметрия, большая часть наблюдений кучкуется в одном месте и есть несколько выделяющихся значений.Фактически это будет влиять на среднее так же, как и выбросы, поэтому при расчете будем брать медиану.

In [ ]:
import pingouin as pg

pg.normality(big_df.query('rev>0'), dv='rev', group='grp', method='normaltest')

#### 1.2. требование гомогенности дисперсий

In [ ]:
pg.homoscedasticity(big_df.query('rev>0'), dv='rev', group='grp')

**Вывод:** Требования не выполнены, но т.к. выборки в несколько сотен наблюдений, то отклонения от нормальности не так критичны. (нормальность выборочного распределения по ЦПТ и является критичной для t-теста).

In [ ]:

b_mean = big_df.query('rev>0 & grp=="B"')['rev'].median()
a_mean = big_df.query('rev>0 & grp=="A"')['rev'].median()

percent = np.round((b_mean-a_mean)/a_mean*100,2)
"Результат между тестом и контролем (в медианах) составил " + str(percent) + "%"

## Вывод: <br> 
Все тесты (u-test,t-test, bootstrap) показали статистически значимые результаты. Исходя из условия задачи (попытка сравнить средние генеральной совокупности) нам  больше подходит t-test т.к. бутстреп создает среднее не из генеральной совокупности, а из выборки.

## Ответы на вопросы (задание 1):

**На какие метрики Вы смотрите в ходе анализа и почему?** <br/>
1) Активные пользователи. Нас интересуют только те кто заходил на платформу и\или оплачивали во время эксперимента. Остальных юзеров из базы мы не учитываем. <br/>
2) CR - конверсия в покупку. Важно понимать конвертит ли новая механика в покупку лучше или хуже. <br/>
3) ARPU, ARPPU - помогает оценить лояльность клиентов, измерить ценность внесенных изменений в продукт. <br/>
4) MRPPU - в эксперименте учавствует вся клиентская база, данные не кластеризованы, поэтому для описания центральной тенденции лучше использоваать медиану. <br/>
**Имеются ли различия в показателях и с чем они могут быть связаны?** <br/>
Разница в CR% между группами незначительна,составляет 0,56 и связана с естественной флуктуацией. 
Различия в ARPU,ARPPU, MRPPU k значительны механики оплаты отличаются между собой количеством тарифов и в новой механике смещение идет в строну увеличение чека. Следовательно, новая механика может сильно повлиять на эти показатели.<br/>
**Являются ли эти различия статистически значимыми?** <br/>
CR - как показали результаты теста хи-квадрат, различия не являются стат.значимыми <br/>
MRPPU - во всех трех тестах различия стат.значимы. Из условия задачи ориентируемся на t-test <br/>
**Стоит ли запускать новую механику на всех пользователей?** <br/>
Стоит, различия в CR не являются стат.значимыми и даже если мы допустили ошибку первого рода, все равно изменение слишком незначительно (-0.08 п.п.). При этом различия между тестом и контролем в медианных чеках составил 35,5%.

## Задание 2. SQL

In [ ]:
import pandahouse as ph

In [ ]:
# Настройка подвлючения к clickhouse
connection_default = {'host': 'http://clickhouse.beslan.pro:8080',
                      'database':'default',
                      'user':'student', 
                      'password':'dpo_python_2020'
                     }

### 2.1 Очень усердные ученики.

In [ ]:
q = '''
SELECT
    uniqExact(st_id) AS hardworking
FROM    
    (
    SELECT
        st_id,
        date_local,
        SUM(correct) AS score
    FROM    
        (
        SELECT 
            st_id,
            correct,
            toStartOfMonth((timest)) AS date_local
        FROM 
            default.peas
        )
    GROUP BY st_id,date_local
    )
HAVING score>=20 
    '''

In [ ]:
q_count_good_st = ph.read_clickhouse(query=q,connection=connection_default)

#### Ответ 2.1.:

In [ ]:
q_count_good_st

### 2.2 Оптимизация воронки

In [ ]:
q_task_2_test = '''
SELECT
    grp,
    arpu,
    arpau,
    cr,
    cr_active,
    cr_math
FROM
    (
    SELECT
        main_table.grp,
        dim_users.all_users,
        main_table.users,
        main_table.money,
        main_table.money_AU,
        main_table.active_users,
        main_table.active_math_users,
        main_table.buyers,
        main_table.math_buyer,
        main_table.active_paying_user,
        divide(buyers,all_users)*100 AS cr,
        divide(active_paying_user,active_users)*100 AS cr_active,
        divide(math_buyer,active_math_users)*100 AS cr_math,
        divide(money,all_users) AS arpu,
        divide(money_AU,active_users) AS arpau
    FROM
        (
        SELECT
            grp,
            uniqExact(st_id) AS users,
            SUM(money) AS money,
            SUM(active_math_true) AS active_math_users,
            SUM(active_user) AS active_users,
            SUM(cr) AS buyers,
            SUM(math_purchase) AS buyers_math,
            SUM(money_active_user) AS money_AU,
            SUM(math_buyer) AS math_buyer,
            SUM(active_paying_user) AS active_paying_user
        FROM
            (
            SELECT 
                l.st_id,
                r.test_grp AS grp,
                l.score_total,
                l.active_math_true,
                l.active_user,
                l.cr,
                l.money,
                l.math_purchase,
                CASE WHEN active_user = 1 THEN money ELSE 0 END AS money_active_user,
                CASE WHEN active_math_true = 1 AND math_purchase = 1 THEN 1 ELSE 0 END AS math_buyer,
                CASE WHEN active_user = 1 AND money>0 THEN 1 ELSE 0 END AS active_paying_user
            FROM
                (
                    SELECT
                        CASE WHEN empty(scoring_table.st_id) THEN sales_table.st_id ELSE scoring_table.st_id END AS st_id,
                        scoring_table.score_total AS score_total,
                        scoring_table.active_math_true AS active_math_true,
                        scoring_table.active_user AS active_user,
                        sales_table.cr AS cr,
                        sales_table.money AS money,
                        sales_table.math_purchase AS math_purchase
                    FROM    
                        (
                        SELECT 
                        st_id,
                        SUM(score) AS score_total,
                        MAX(active_math_true) AS active_math_true,
                        CASE WHEN score_total>10 THEN 1 ELSE 0 END AS active_user
                        FROM
                            (
                            SELECT
                                st_id,
                                subject,
                                score,
                                active_math_true
                            FROM
                                (
                                SELECT 
                                    st_id,
                                    subject,
                                    SUM(correct) AS score,
                                    CASE WHEN subject ='Math' AND score >=2 THEN 1 ELSE 0 END AS active_math_true
                                FROM    
                                    (
                                    SELECT 
                                        st_id,
                                        correct,
                                        subject
                                    FROM default.peas
                                    )
                                GROUP BY st_id,subject
                                )
                            )
                        GROUP BY st_id
                        ) AS scoring_table
                    FULL OUTER JOIN 
                        (
                        SELECT 
                            st_id,
                            SUM(money) AS money,
                            MAX(converted) AS cr,
                            MAX(math_purchase) AS math_purchase
                        FROM 
                            (
                            SELECT
                                st_id,
                                subject,
                                SUM(money) AS money,
                                materialize(1) AS converted,
                                CASE WHEN subject ='Math' AND money> 0 THEN 1 ELSE 0 END AS math_purchase
                            FROM
                                (
                                SELECT 
                                    st_id,
                                    toDate(sale_time) AS date_local,
                                    money,
                                    subject
                                FROM default.final_project_check
                                )
                            GROUP BY st_id, subject
                            )
                        GROUP BY st_id
                        ) AS sales_table 
                    ON scoring_table.st_id = sales_table.st_id 
                ) AS l
        LEFT JOIN 
            (
            SELECT *
            FROM default.studs
            ) AS r 
            ON l.st_id = r.st_id
            )
            GROUP BY grp
        ) AS main_table
    LEFT JOIN 
        (
        SELECT
            test_grp AS grp,
            COUNT(DISTINCT st_id) AS all_users
        FROM
        default.studs
        GROUP BY test_grp
        ) AS dim_users
    ON main_table.grp =  dim_users.grp
    )
    
'''

In [ ]:
#С помощью join split удаляем пробелы и переносы строки, чтобы уменьшить длину запроса
result_task_2_test = ph.read_clickhouse(query=" ".join(q_task_2_test.split()),connection=connection_default)


#### Ответ 2.2.:

In [ ]:
result_task_2_test

## Задание 3.

#### Задача 3.1. Решение

In [ ]:
def get_groups_add():
        try:
            # Получаем доп.таблицу с группами
            # используем api 
            base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?' 
            public_key = 'https://disk.yandex.ru/d/5Kxrz02m3IBUwQ' 
            # получаем url 
            final_url = base_url + urlencode(dict(public_key=public_key)) 
            response = requests.get(final_url) 
            download_url_orders = response.json()['href'] 
            # загружаем файл в df 
            download_response = requests.get(download_url_orders) 
            groups_add_task3 = pd.read_csv(download_url_orders, sep=',')
        except:
            path = 'file_groups_add_error.csv'
            groups_add_task3 = pd.read_csv(path, sep=',')
        groups_full = pd.DataFrame( np.concatenate( (groups_df.values, groups_add_task3.values), axis=0 ) )
        groups_full.columns = [ 'id', 'grp']
        groups_full.drop_duplicates()
        # Создаем таблицу всех юзеров с группой тестирования и инфой об активности.
        all_users = pd.merge(groups_full, active_studs_df, how ='left', left_on='id', right_on='student_id').drop('student_id', axis=1)
        big_df = pd.merge(checks_df, all_users, how ='outer',left_on='student_id', right_on='id').drop('student_id', axis=1)
        big_df = big_df.query('info=="были на платформе" or rev>0').set_index('id')
        big_df['rev'] = big_df['rev'].fillna(0)
        # Создаем столбец -лейбл, который принимает значение 1, если пользователь совершил покупку и 0 если покупку не совершал.
        big_df['converted'] = np.where(big_df.rev>0,1,0)
        groups = big_df.reset_index().groupby(['grp'])\
                .agg(\
                    ids = ('id','size'),\
                    revenue = ('rev','sum'),\
                    converted =('converted','sum')\
                    )
        # Добавляем столбец с MRPPU
        groups = pd.concat([groups, mrppu_by_groups], axis=1)
        #  Считаем клиентов, которые не делали покупок
        groups['unconverted'] = groups['ids']-groups['converted']
        # Считаем % клиентов, которые не делали покупок
        groups['part_unconv_%'] = np.round(((groups['unconverted']/groups['ids'])*100),2)
        # Считаем CR% 
        groups['cr_%'] = round(((groups.converted/groups.ids)*100),2)
        groups['revenue'] = np.round((groups.revenue),2)
        # # Посчитаем разницу в CR между группами в %
        groups['diff%_cr_B-A'] = np.round(((groups.loc['B']['cr_%'] - groups.loc['A']['cr_%'])/groups.loc['A']['cr_%']),2)
        # Посчитаем разницу в ARPU между группами абсолют. и в %
        groups['ARPU'] = np.round((groups.revenue/groups.ids),2)
        groups['diff%_ARPU_B-A'] = np.round((((groups.loc['B']['ARPU'] - groups.loc['A']['ARPU']) / groups.loc['A']['ARPU'])*100),2)
        # Посчитаем разницу в ARPPU между группами абсолют. и в %
        groups['ARPPU'] = np.round((groups.revenue/groups.converted),2)
        # groups['diff_ARPPU_B-A'] = np.round((groups.loc['B']['ARPPU'] - groups.loc['A']['ARPPU']),2)
        groups['diff%_ARPPU_B-A'] = np.round((((groups.loc['B']['ARPPU'] - groups.loc['A']['ARPPU']) / groups.loc['A']['ARPPU'])*100),2)
        groups['diff%_MRPPU_B-A'] = np.round((((groups.loc['B']['MRPPU'] - groups.loc['A']['MRPPU']) / groups.loc['A']['MRPPU'])*100),2)
        # Посчитаем % клиентов в группах
        groups['%_clients'] = np.round(((groups['ids']/(groups.loc['A']['ids'] + groups.loc['B']['ids']))*100),2)
        f_one_result = groups.reset_index()[
            ['grp','%_clients','cr_%','ARPU','ARPPU','MRPPU','diff%_cr_B-A','diff%_ARPU_B-A','diff%_ARPPU_B-A','diff%_MRPPU_B-A']]
        return f_one_result

#### Задача 3.1. Результат

In [ ]:
get_groups_add()

#### Задача 3.2. Решение

In [ ]:
f_one_result = get_groups_add()

In [ ]:
def visuals_total(param):
    param = param[[ 'diff%_cr_B-A','diff%_ARPU_B-A','diff%_ARPPU_B-A','diff%_MRPPU_B-A']]
    param = param.stack().to_frame().reset_index().query('level_0==0')\
                    .rename(columns={'level_0':'index', 'level_1':'metric'})
    param = param.rename(columns= {param.columns[2]:'B_vs_A_%'})
    param['A'] = 100 + a_vs_b_deviation['B_vs_A_%']
    param['B'] = 100 - a_vs_b_deviation['B_vs_A_%']
    param = param[['metric','A','B']].set_index('metric').plot(kind='bar', stacked=False, color=['#4c72b0', '#dd8452']);
    return param

#### Задача 3.2. Результат

In [ ]:
visuals_total(f_one_result)